<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/RtGender-Notebooks/Removed_Irrelevant_Mixed/RtGender_Annotations_Sentiment_Back_Translation_Removed_Irrelevant_%26_Mixed_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RtGender - Annotations - Sentiment - Back Translation - Removed Irrelevant & Mixed Sentiment
---
[Source](https://github.com/bhadreshpsavani/ExploringSentimentalAnalysis/blob/main/SentimentalAnalysisWithDistilbert.ipynb)




In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


<a id='section01'></a>
## Load Modules

In [2]:
%%capture
#!pip install transformers==4.1.1 # needed for Back translation but upgrade for model run

# # Only needed for Back translation
# !pip install sentencepiece==0.1.94
# !pip install mosestokenizer==1.1.0
# from transformers import MarianMTModel, MarianTokenizer

!pip install -q transformers
# needed for tokenization
#

In [3]:
%%capture
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [4]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn import metrics
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import logging
logging.basicConfig(level=logging.ERROR)

In [5]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
# # Setting up the device for TPUs
# %%capture
# # https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=3P6b3uqfzpDI
# import os
# assert os.environ['COLAB_TPU_ADDR'] 
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# # imports the torch_xla package
# VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

# import torch_xla
# import torch_xla.core.xla_model as xm

# # Creates a random tensor on xla:1 (a Cloud TPU core)
# device = xm.xla_device()



<a id='section02'></a>
## Import and Reshape Data

In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/w266/train_oversampled.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_dev.csv')

print('train_shape: ',train_df.shape)
print('dev_shape: ',dev_df.shape)

train_shape:  (21184, 9)
dev_shape:  (2303, 9)


In [8]:
# there are NaNs in the dev dataset remove 
nan_values = dev_df[dev_df.isna().any(axis=1)] 
print(nan_values)

# return without missing values in response_text
dev_df.dropna(subset = ["response_text"], inplace=True)

print("Train shape", train_df.shape)
print("Dev shape", dev_df.shape)

      Unnamed: 0         source op_gender  ...   relevance label labels_4
830         2576  facebook_wiki         M  ...  Irrelevant     1        1
1664        2722  facebook_wiki         W  ...  Irrelevant     1        1

[2 rows x 9 columns]
Train shape (21184, 9)
Dev shape (2301, 9)


In [9]:
# remove mixed sentiment and irrelevant examples
train_df  = train_df[train_df['relevance'] != 'Irrelevant']
dev_df = dev_df[dev_df['relevance'] != 'Irrelevant']
train_df  = train_df[train_df['sentiment'] != 'Mixed']
dev_df = dev_df[dev_df['sentiment'] != 'Mixed']

print('updated train_shape: ',train_df.shape)
print('updated dev_shape: ',dev_df.shape)
print("Unique sentiments: ", train_df['sentiment'].unique())

updated train_shape:  (13576, 9)
updated dev_shape:  (1821, 9)
Unique sentiments:  ['Positive' 'Neutral' 'Negative']


## Process Data

### Determine Max Length

In [10]:
# find the P99 of length for response_text and set that as the max length 
max_length = train_df['response_text'].astype(str).map(len).quantile(0.99).astype(int)
print(f"99th %tile of response_text length: {max_length}")

99th %tile of response_text length: 286


## Back Translate (skip after first run)
[source](https://dzlab.github.io/dltips/en/pytorch/text-augmentation/)

In [ ]:
# Helper function to download data for a language
def download(model_name):
  tokenizer = MarianTokenizer.from_pretrained(model_name)
  model = MarianMTModel.from_pretrained(model_name)
  return tokenizer, model


# model that can translate from English to Romance languages
# this is a single model that can translate to any of the romance languages

target_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')
target_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')

# initialize models that can translate Romance languages to English.

en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ROMANCE-en')
en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ROMANCE-en')

Beam search was timing out even with Colab Pro+. Setting a max beam eval of 2 per this [stackoverflow] (https://github.com/huggingface/transformers/issues/7324).

In [ ]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(i) for i in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,return_tensors="pt",
                                              max_length = 150, 
                                              eval_beams=1, 
                                              eval_max_gen_length=150 
                                              )
                                       
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [ ]:
def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

In [ ]:
#@title
# note that even a single text needs to be inside a list
# single_sentence = ['I would like to have a cup of coffee']

# aug_texts = back_translate(single_sentence, source_lang="en", target_lang="es")
# print(aug_texts)

### Test timing for Back translation

In [ ]:
#import timeit

# train_df_aug = train_df.drop(['Unnamed: 0', 'source', 'op_gender', 'post_text', 'sentiment', 'relevance', 'label'], axis = 1)
# train_df_aug = train_df_aug[:10]


Time to run 10 examples using ColabPro+ with truncation at 99% of response_text length: 
* TPU: 1 loop, best of 5: 40.4 s per example
* GPU: 1 loop, best of 5: 31.2 s per example

But when I truncate to 150: 1 loop, best of 5: 14.5 s per example


In [ ]:
# %%timeit
# es = []
# for i in train_df_aug['response_text']:
#   es.append(back_translate([i], source_lang='en', target_lang='es')[0])


### Run back Translation once.. 
save out and import for future runs.
only one language ran in 24 hours. Will back translate Spanish only

In [ ]:
## run the first time only
train_df_aug = train_df.drop(['Unnamed: 0', 'source', 'op_gender', 'post_text', 'sentiment', 'relevance', 'label'], axis = 1)

train_df_aug['es_trans'] = train_df_aug['response_text'].apply(lambda x: back_translate([x], source_lang='en', target_lang='es')[0])
train_df_aug['fr_trans'] = train_df_aug['response_text'].apply(lambda x: back_translate([x], source_lang='en', target_lang='fr')[0])
train_df_aug['it_trans'] = train_df_aug['response_text'].apply(lambda x: back_translate([x], source_lang='en', target_lang='it')[0])
train_df_aug['pt_trans'] = train_df_aug['response_text'].apply(lambda x: back_translate([x], source_lang='en', target_lang='pt')[0])
train_df_aug['ro_trans'] = train_df_aug['response_text'].apply(lambda x: back_translate([x], source_lang='en', target_lang='ro')[0]) 
train_df_aug.to_csv('/content/drive/MyDrive/w266/back_translation_augmented_train_data.csv', index=False)

## Convert to HuggingFace format

In [11]:
#import 
train_df_aug = pd.read_csv('/content/drive/MyDrive/w266/back_translation_augmented_train_data.csv')
# combine the translated (for the basic train dataset) with the oversampled version
train_df_aug = train_df.merge(train_df_aug, on= ["labels_4", "response_text"], how='left', indicator=True)
print('Back translated train shape: ', train_df_aug.shape)


Back translated train shape:  (14218, 15)


In [12]:
# reshape data
train_augmented_df = pd.concat([
  train_df_aug[['response_text','labels_4']],
  train_df_aug[['es_trans','labels_4']].rename(columns={'es_trans':'response_text'}),
  train_df_aug[['fr_trans','labels_4']].rename(columns={'fr_trans':'response_text'}),
  train_df_aug[['it_trans','labels_4']].rename(columns={'it_trans':'response_text'})
  # train_df_aug[['pt_trans','labels_4']].rename(columns={'pt_trans':'response_text'}),
  # train_df_aug[['ro_trans','labels_4']].rename(columns={'ro_trans':'response_text'})
  ])

In [13]:
print('Back translated train shape: ', train_augmented_df.shape)
print(train_augmented_df['labels_4'].value_counts(normalize=True))


Back translated train shape:  (56872, 2)
2    0.381066
0    0.332466
1    0.286468
Name: labels_4, dtype: float64


In [14]:
# change to dataset to work with Huggingface transformer & remove unused columns
columns_to_remove = ['op_gender', 'source', 'Unnamed: 0', 'relevance', 'sentiment','post_text','label']

from datasets import load_dataset
train_dataset = Dataset.from_pandas(train_augmented_df)
dev_dataset = Dataset.from_pandas(dev_df)

dev_dataset = dev_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= '__index_level_0__')


# rename labels_4 to labels
train_dataset = train_dataset.rename_column("labels_4", "label")
dev_dataset = dev_dataset.rename_column("labels_4", "label")

In [15]:
# combine into a DataDictionary for huggingface use
rtg_dataset = DatasetDict({
    'train': train_dataset,
    'dev': dev_dataset 
})

rtg_dataset

DatasetDict({
    train: Dataset({
        features: ['response_text', 'label', '__index_level_0__'],
        num_rows: 56872
    })
    dev: Dataset({
        features: ['response_text', 'label'],
        num_rows: 1821
    })
})

## Tokenize

In [16]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length = int(max_length))

def tokenize(batch):
    return tokenizer(batch["response_text"], padding=True, truncation=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [17]:
rtg_encoded = rtg_dataset.map(tokenize, batched=True, batch_size=None)
rtg_encoded['train'].features

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'__index_level_0__': Value(dtype='int64', id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'response_text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

# Model

In [18]:
import gc
try:
  del trainer
  del results
  del cr
  del model
  gc.collect()
  torch.cuda.empty_cache()
except: pass



In [19]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
epochs = 2
iterations = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [20]:
rtg_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [21]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average = 'macro')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1_weighted, "f1_macro": f1_macro} 

In [22]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(rtg_encoded["train"]) // batch_size*3
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                 # metric_for_best_model="f1_macro",
                                 # weight_decay=0.01,
                                  evaluation_strategy="steps",
                                  #save_strategy="steps",
                                  disable_tqdm=False
                                  )

In [23]:
from sklearn.metrics import classification_report

accuracy_list = []
weighted_f1_score_list = []
macro_f1_score_list = []
negative_f1_score = []
neutral_f1_score = []
positive_f1_score = []


for i in range(iterations):
  try:
    del trainer
    del results
    del cr
  except: pass


  trainer = Trainer(model=model, 
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=rtg_encoded["train"],
                    eval_dataset=rtg_encoded["dev"]
                    )
  trainer.train()
  results = trainer.evaluate()

  # append macro metrics to lists
  accuracy_list.append(results.get('eval_accuracy'))
  weighted_f1_score_list.append(results.get("eval_f1"))
  macro_f1_score_list.append(results.get("eval_f1_macro"))

  trainer.predict(rtg_encoded["dev"])
  # append the class-level F1 scores
  outputs = trainer.predict(rtg_encoded["dev"])
  predictions = outputs.predictions.argmax(1)
  labels = rtg_encoded["dev"]['label']
  cr = classification_report(labels, predictions, digits=3, output_dict=True)
  negative_f1_score.append(cr.get('0').get("f1-score"))
  neutral_f1_score.append(cr.get('1').get("f1-score"))
  positive_f1_score.append(cr.get('2').get("f1-score"))


  print(f'---------------------------Iteration {i+1} Complete---------------------------\n')

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, response_text.
***** Running training *****
  Num examples = 56872
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14218


Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.730100,0.601371,0.754530,0.749227,0.678373
1000,0.580600,0.678477,0.753981,0.752951,0.688737
1500,0.501600,0.695667,0.770456,0.771259,0.714728
2000,0.467200,0.735842,0.757825,0.761645,0.698929
2500,0.402700,0.738761,0.766612,0.759342,0.694681
3000,0.372700,0.832406,0.770456,0.765984,0.703567
3500,0.346800,1.006674,0.752883,0.754757,0.691438
4000,0.309800,0.998967,0.762219,0.759429,0.695154
4500,0.281900,1.102897,0.762219,0.759286,0.693643
5000,0.256700,1.066010,0.750686,0.749997,0.687097


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 1821
  Batch size = 8
Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 1821
  Batch size = 8
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation **

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, response_text.
***** Running training *****
  Num examples = 56872
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14218


---------------------------Iteration 1 Complete---------------------------



...

---------------------------Iteration 2 Complete---------------------------



Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.124100,1.547889,0.753981,0.756410,0.695550
1000,0.232700,1.354831,0.744097,0.728273,0.653264
1500,0.481700,0.748926,0.764415,0.766797,0.709906
2000,0.412600,0.895526,0.756178,0.756904,0.692382
2500,0.358500,0.871552,0.756727,0.742645,0.670836
3000,0.323200,1.007928,0.757825,0.757996,0.691575
3500,0.315200,1.080027,0.757825,0.756914,0.692917
4000,0.261000,1.158891,0.758375,0.754162,0.689649
4500,0.246000,1.150684,0.761669,0.759064,0.694852
5000,0.223800,1.108647,0.756727,0.753789,0.687617


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 1821
  Batch size = 8
Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 1821
  Batch size = 8
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation **

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


---------------------------Iteration 3 Complete---------------------------



## Evaluate

In [24]:
import statistics

print("%15s %s (%s)" % ("","Mean", "StDev"))

print("-"*29)
print("Macro Scores")
print("-"*29)

print(f"%15s %s " %("Accuracy",
    round(statistics.mean(accuracy_list),3)))
    # round(statistics.stdev(accuracy_list),3))

print(f"%15s %5s " %("Macro F1",
    round(statistics.mean(macro_f1_score_list),3)))
    # round(statistics.stdev(macro_f1_score_list),3)))
print(f"%15s %5s " %("Weighted F1",
    round(statistics.mean(weighted_f1_score_list),3)))
    # round(statistics.stdev(weighted_f1_score_list),3)))

print("-"*29)
print("Class Scores")
print("-"*29)

print(f"%15s %s " %("Positive",
    round(statistics.mean(positive_f1_score),3)))
    # round(statistics.stdev(positive_f1_score),3)))
print(f"%15s %5s " %("Neutral",
    round(statistics.mean(neutral_f1_score),3)))
    # round(statistics.stdev(neutral_f1_score),3)))
print(f"%15s %5s " %("Negative",
    round(statistics.mean(negative_f1_score),3)))
    # round(statistics.stdev(negative_f1_score),3)))


                Mean (StDev)
-----------------------------
Macro Scores
-----------------------------
       Accuracy 0.759 
       Macro F1 0.688 
    Weighted F1 0.754 
-----------------------------
Class Scores
-----------------------------
       Positive 0.865 
        Neutral 0.551 
       Negative 0.649 


In [25]:
output_model_file = '/content/drive/MyDrive/w266/backtranslate_bert_rtgender_sentiment_removed_irrelevant_mixed.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved
